This jupyter notebook sets out about how to create a chat bot capable of python coding which uses data from a csv file. Below we have done the usual set up, if you have not previously run pip install openai within the jupyter notebook folder you are using or get any error messages relating to "openai" not being correct and have not run this first line run it without the "#" at the beginning and restart the notebook. Note: do not press restart and run all as this will cause the chat bot to interupt itself, go through the boxes one by one.

In [ ]:
#pip install openai

In [ ]:
from openai import OpenAI

openai_api_key = ""

client = OpenAI(api_key=openai_api_key)

Next we'll create the assistant as we have done before:

In [ ]:
assistant = client.beta.assistants.create(
    instructions = "You are an automatic python coder, when given a set of data you write and run code to plot the data, when you are given a prompt to code you write and run that code",
    name = "Python Plotter",
    model = "gpt-4o",
    tools = [{"type": "code_interpreter"}],
)

Then we'll create an object which contains the file we want to use and a marked "purpose". Purposes are a defined list and are marked in the open ai website when we upload our file. These are the list of possible purpose tags: 'fine-tune', 'assistants', 'batch', 'user_data', 'responses', 'vision'

After that we create a thread as we have done before including the file in the attachments. Two things we can note here: one the reason we do not need to turn this file into a vector store is because our chat bot is not using the document as a knowledge base, instead like a prompt this is an input for the chat bot. second we could have instead chose to upload the file when creating the assistant by including the code marked by a # at the bottom of the below box, however each time we run the above code a new assistant is created, which is not ideal, whereas if we run it through the thread creation we use the same assistant and it mimics how someone would upload a file to an interface of an already existing assistant. Note: if you ever want to reset the conversation you are having with the chatbot you can run the below box again and the assistant will be maintained but the conversation will restart.

In [ ]:
file = client.files.create(
    file=open("x_squared_data.csv", "rb"),
    purpose="user_data"
)

thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "I need to create a line plot using this data, can you help me?",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

#tool_resources={
    #"code_interpreter": {
      #"file_ids": [file.id]
       # }
   # }


Next again we'll write our event handler which actually allows our stream function to print text. The three new imports shown here are PIL, io and requests. PIL allows us to show images in jupyter notebooks (this means this might be unnescerary for future versions where you're using different UI), io allows the raw image data to be read and converted, requests allows for the download of the image data from the chat bot. There are two new sections of code within the following box: that is the new elif statement in the event handler's on_tool_call_delta_function, this elif is there for the case when the output of the chat bot is an image and it uses the new download_image function which grabs the image from the url and downloads it to display the image.

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler
from PIL import Image
import io
import requests

class EventHandler(AssistantEventHandler):    
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)
      
    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)
      
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)
  
    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)
                    elif output.type == "image":
                        # Fetch the image data using the file_id
                        file_id = output.image.file_id
                        image_data = self.download_image(file_id)
                        if image_data:
                            image = Image.open(io.BytesIO(image_data))
                            image.show()
  
    def download_image(self, file_id):
        url = f"https://api.openai.com/v1/files/{file_id}/content"
        headers = {
            "Authorization": f"Bearer {openai_api_key}",
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Failed to download image: {response.status_code} {response.text}")
            return None

Then as we have done previously we have the stream run from the chat bot making sure to specificy the thread id, assistant id, instructions and event handler that we want to use

In [ ]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please be calm and understanding of any requests",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

As you can see the outputted data is a quadratic plot with values at integers of x, but what if we wanted to swap the x and y axis? We can use the following code, below we create a "thread message", this adds a message onto the already existing thread using the already existing thread id, we then run our stream again and we get a new out put. You can play with this swapping the content=input input out for different things or you could even go back to the beginning and put in a new cvs file to get an entirely new plot.

In [ ]:
thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="please swap the x and y axis"
)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please be calm and understanding of any requests",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

Below this are several other boxes of examples of what you could have this do in the same thread.

In [ ]:
thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="please make a y = x cubed graph ignoring the previous data"
)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please be calm and understanding of any requests",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

In [ ]:
thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="please write code for calculating the area of a circle of radius 1 using the monte carlo method"
)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please be calm and understanding of any requests",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()